## 1. Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltd
import seaborn as sns

from gensim.models import Word2Vec

C:\Users\JAVI\Miniconda3\envs\dhds\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 2. Cargar datos

In [2]:
movies=pd.read_csv('../Data/movies.csv')
ratings=pd.read_csv('../Data/ratings.csv')
links=pd.read_csv('../Data/links.csv')
tags=pd.read_csv('../Data/tags.csv')
imdb=pd.read_csv('../Data/tab_imdb_info.csv',index_col=0)

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
links.dtypes

movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

In [6]:
imdb.head()

,actors,country,director,req_imdbid,year
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995


In [7]:
imdb.head()

,actors,country,director,req_imdbid,year
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995


## 3. Feature Extraction

Corregir id's de imdb:

In [8]:
imdb['imdbId']=imdb['req_imdbid'].apply(lambda x: x[2:]).astype('int64')

In [9]:
pd.merge(links,movies,on='movieId').head()

,movieId,imdbId,tmdbId,title,genres
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,Father of the Bride Part II (1995),Comedy


In [10]:
imdb.isna().sum()

actors        32
country       12
director      94
req_imdbid     0
year           0
imdbId         0
dtype: int64

In [11]:
imdb.fillna('-',inplace=True)

In [12]:
imdb['actors'].apply(lambda x: len(x.split(','))).unique()

array([4, 2, 3, 1], dtype=int64)

In [13]:
# imdb['director1'] = imdb['director'].apply(lambda x: x.split(',')[0])
# imdb['actor1'] = imdb['actors'].apply(lambda x: x.split(',')[0])
# #imdb['actor2'] = imdb['actors'].apply(lambda x: x.split(',')[1])
# imdb['country1'] = imdb['country'].apply(lambda x: x.split(',')[0])

Creo un dataframe con todos los datos necesarios para crear features por películas:

In [14]:
mov_with_feat = pd.merge(links,imdb,on='imdbId')
mov_with_feat = pd.merge(mov_with_feat,movies,on='movieId')
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres
0,1,114709,862.0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy


In [15]:
mov_with_feat.isna().sum()

movieId       0
imdbId        0
tmdbId        8
actors        0
country       0
director      0
req_imdbid    0
year          0
title         0
genres        0
dtype: int64

In [16]:
mov_with_feat.fillna('-',inplace=True)

In [17]:
mov_with_feat['actors'] = mov_with_feat['actors'].str.replace(' ','')
mov_with_feat['genres'] = mov_with_feat['genres'].str.replace('|',',')
mov_with_feat['genres'] = mov_with_feat['genres'].str.replace(' ','')
mov_with_feat['director'] = mov_with_feat['director'].str.replace(' ','')
mov_with_feat['country'] = mov_with_feat['country'].str.replace(' ','')
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres
0,1,114709,862,"TomHanks,TimAllen,DonRickles,JimVarney",USA,JohnLasseter,tt0114709,1995,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy"
1,2,113497,8844,"RobinWilliams,JonathanHyde,KirstenDunst,Bradle...",USA,JoeJohnston,tt0113497,1995,Jumanji (1995),"Adventure,Children,Fantasy"
2,3,113228,15602,"WalterMatthau,JackLemmon,SophiaLoren,Ann-Margret",USA,HowardDeutch,tt0113228,1995,Grumpier Old Men (1995),"Comedy,Romance"
3,4,114885,31357,"WhitneyHouston,AngelaBassett,LorettaDevine,Lel...",USA,ForestWhitaker,tt0114885,1995,Waiting to Exhale (1995),"Comedy,Drama,Romance"
4,5,113041,11862,"SteveMartin,DianeKeaton,MartinShort,KimberlyWi...",USA,CharlesShyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy


Armado de tokens:

In [18]:
#df1 = mov_with_feat[['actor1','director1','country1']]
df1 = mov_with_feat[['actors','director','country','genres']]
df1 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)
df1 = df1.str.replace(' ','')
df1 = pd.DataFrame({'clean': df1}) 
print(df1.iloc[0].values)
sent = [row.split(',') for row in df1['clean']]
mov_sent = [a for a in zip(imdb['req_imdbid'],sent)]
mov_sent[0:2]

['TomHanks,TimAllen,DonRickles,JimVarney,JohnLasseter,USA,Adventure,Animation,Children,Comedy,Fantasy']


[('tt0114709',
  ['TomHanks',
   'TimAllen',
   'DonRickles',
   'JimVarney',
   'JohnLasseter',
   'USA',
   'Adventure',
   'Animation',
   'Children',
   'Comedy',
   'Fantasy']),
 ('tt0113497',
  ['RobinWilliams',
   'JonathanHyde',
   'KirstenDunst',
   'BradleyPierce',
   'JoeJohnston',
   'USA',
   'Adventure',
   'Children',
   'Fantasy'])]

Entrenar modelo de word2vec de gensim:

In [19]:
model = Word2Vec(sent, min_count=1,size= 50,workers=3, window =3, sg = 1)

In [20]:
#Representation for director John Lasseter
model.wv['JohnLasseter']

array([-0.00530838, -0.00821753, -0.01241069, -0.00390607,  0.02467749,
        0.00081867,  0.12036797, -0.00406078, -0.06494071, -0.1071832 ,
        0.00553983, -0.04288299, -0.05797353, -0.01357377,  0.05264683,
        0.03802342, -0.0108917 , -0.0040256 , -0.06203947, -0.05294405,
        0.08883951, -0.16014571, -0.02038274, -0.05808151, -0.08904957,
        0.01099537,  0.01851519, -0.07417219,  0.03949223, -0.00190608,
       -0.0323001 ,  0.02179354,  0.01100896, -0.0589769 , -0.0895072 ,
       -0.00086337,  0.07297938,  0.08725852,  0.00248522,  0.03030696,
        0.00627305,  0.01637614,  0.03988529,  0.04127919, -0.10453401,
        0.05467838, -0.02688392,  0.00482532, -0.02741257, -0.00327732],
      dtype=float32)

In [21]:
#Get similarity
model.wv.similarity('TomHanks', 'USA')

0.5196904062463137

In [22]:
model.wv.similarity('JohnnyDepp','TimBurton')

0.8929011892125337

In [23]:
model.wv.similarity('Action','USA')

0.9708765284791

In [24]:
model.wv.similarity('Children','Comedy')

0.9092230750871284

In [25]:
model.wv.similarity('Action','Adventure')

0.971513454660109

In [26]:
model.wv['Adventure']

array([ 0.37382078, -0.56967914,  0.12922356, -0.0240774 ,  0.43953124,
        0.14738359,  0.65133375,  0.00728319,  0.0576599 , -1.0973365 ,
       -0.2244763 , -0.6371443 , -0.65482706, -0.12907426, -0.14620261,
        0.28750563, -0.01033304,  0.7774263 , -0.9040231 , -0.36122504,
        0.2721634 , -1.1640457 , -0.26613644,  0.14863424, -0.3163089 ,
        1.0896744 , -0.31850117,  0.5150338 , -0.3330369 , -0.08649182,
        0.08062632, -0.240044  ,  0.7171102 , -0.10483798, -0.34158668,
       -0.7336749 , -0.214322  ,  0.17189251,  0.1103375 ,  0.18603888,
       -0.0085896 ,  0.14359958,  0.40312493,  0.71055615, -0.46122903,
        0.16732833, -0.39912406,  0.32370317,  0.25611362, -0.07388034],
      dtype=float32)

In [27]:
model.wv['(nogenreslisted)']

array([ 0.01119771, -0.04540184, -0.00468265,  0.00206574,  0.06557903,
        0.01310313,  0.23003586, -0.04145144, -0.10902765, -0.20110203,
        0.02183666, -0.11288605, -0.12378885, -0.01901232,  0.05343007,
        0.0819945 , -0.00415863,  0.02138261, -0.13259575, -0.11114521,
        0.13746491, -0.29116663, -0.01068106, -0.0631005 , -0.15588225,
        0.04977215,  0.02221074, -0.11577576,  0.04281944,  0.03510855,
       -0.06990065,  0.02522622,  0.04540444, -0.07618127, -0.14356816,
       -0.03932277,  0.1387692 ,  0.16024189,  0.05033239,  0.01416309,
       -0.01042143,  0.03254402,  0.06467213,  0.07598737, -0.18424535,
        0.0997132 , -0.03457621,  0.01311972, -0.03643058, -0.03747056],
      dtype=float32)

In [28]:
imdb.head()

,actors,country,director,req_imdbid,year,imdbId
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995,114709
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995,113497
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995,113228
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995,114885
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995,113041


In [29]:
imdb.dtypes

actors        object
country       object
director      object
req_imdbid    object
year          object
imdbId         int64
dtype: object

In [30]:
def rep_col(text):
    #print(text)
    rep = []
    for item in text.split(','):
        rep.append(model.wv[item])
    return np.concatenate(rep,axis=0)

In [31]:
def items_rep(item_lst):
    a=0
    for item in item_lst:
        a+=model.wv[item]
    a/=len(item_lst)
    return a



In [32]:
mov_with_feat['actors_rep'] = mov_with_feat['actors'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['director_rep'] = mov_with_feat['director'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['country_rep'] = mov_with_feat['country'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['genres_rep'] = mov_with_feat['genres'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres,actors_rep,director_rep,country_rep,genres_rep
0,1,114709,862,"TomHanks,TimAllen,DonRickles,JimVarney",USA,JohnLasseter,tt0114709,1995,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy","[-0.012516696, -0.016406087, -0.015700508, 0.0...","[-0.0053083794, -0.00821753, -0.012410687, -0....","[0.27281365, -0.8571311, 0.22393864, -0.194021...","[0.35758302, -0.55831045, 0.10758762, 0.036733..."
1,2,113497,8844,"RobinWilliams,JonathanHyde,KirstenDunst,Bradle...",USA,JoeJohnston,tt0113497,1995,Jumanji (1995),"Adventure,Children,Fantasy","[-0.0071320916, -0.015461, -0.009175457, 0.001...","[0.017497657, -0.015911492, -0.0024551374, 0.0...","[0.27281365, -0.8571311, 0.22393864, -0.194021...","[0.36124364, -0.53867704, 0.09701236, 0.052505..."
2,3,113228,15602,"WalterMatthau,JackLemmon,SophiaLoren,Ann-Margret",USA,HowardDeutch,tt0113228,1995,Grumpier Old Men (1995),"Comedy,Romance","[-0.012997238, -0.0101928115, -0.015151955, 0....","[0.004719363, -0.026426474, -0.0106845265, 0.0...","[0.27281365, -0.8571311, 0.22393864, -0.194021...","[0.28334972, -0.6019492, 0.016440647, -0.05251..."
3,4,114885,31357,"WhitneyHouston,AngelaBassett,LorettaDevine,Lel...",USA,ForestWhitaker,tt0114885,1995,Waiting to Exhale (1995),"Comedy,Drama,Romance","[-0.009106173, -0.010243561, -0.007373129, 0.0...","[-0.013581269, -0.030414926, -0.0047316803, -0...","[0.27281365, -0.8571311, 0.22393864, -0.194021...","[0.26314408, -0.6409857, 0.039903723, -0.07393..."
4,5,113041,11862,"SteveMartin,DianeKeaton,MartinShort,KimberlyWi...",USA,CharlesShyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy,"[-0.014934219, -0.024522327, -0.010537192, -0....","[-0.0062812464, -0.011112935, -0.012702396, 0....","[0.27281365, -0.8571311, 0.22393864, -0.194021...","[0.36966288, -0.68920994, 0.04608947, -0.01302..."


In [33]:
# mov_with_feat['actors_rep'][0]

In [34]:
# mov_with_feat['director_rep'][0]

In [35]:
# mov_with_feat['country_rep'][0]

In [36]:
# np.concatenate([mov_with_feat['actors_rep'][0],mov_with_feat['director_rep'][0],mov_with_feat['country_rep'][0]],axis=0)

In [37]:
mov_with_feat['all_feat']=mov_with_feat.apply(lambda x: np.concatenate([x['actors_rep'],x['director_rep'],x['country_rep'],x['genres_rep']],axis=0),axis=1)

In [38]:
mov_with_feat[['movieId','all_feat']].head()

,movieId,all_feat
0,1,"[-0.012516696, -0.016406087, -0.015700508, 0.0..."
1,2,"[-0.0071320916, -0.015461, -0.009175457, 0.001..."
2,3,"[-0.012997238, -0.0101928115, -0.015151955, 0...."
3,4,"[-0.009106173, -0.010243561, -0.007373129, 0.0..."
4,5,"[-0.014934219, -0.024522327, -0.010537192, -0...."


In [39]:
a = mov_with_feat['all_feat'][0].reshape(1,-1)
for mov_feat in mov_with_feat['all_feat'][1:]:
    b = mov_feat.reshape(1,-1)
    a = np.concatenate([a,b])

a.shape

(9742, 200)

In [40]:
#temp_actor = rep_col(imdb['actors'][0])
#print(np.concatenate(temp_actor,axis=0))
#temp_director = rep_col(imdb['director'][0])
#temp_country = rep_col(imdb['country'][0])
#a = np.concatenate(temp_actor,axis=0)
##np.concatenate([temp_actor,temp_director,temp_country],axis=1)
#np.concatenate(temp_director,axis=0)

In [41]:
#def get_movie_features(fila):
#    temp_actor = rep_col(fila['actors'])
#    a = np.concatenate(temp_actor,axis=0)
#    temp_director = rep_col(fila['director'])
#    b = np.concatenate(temp_director,axis=0)
#    temp_country = rep_col(fila['country'])
#    c = np.concatenate(temp_country,axis=0)
#    feat_lst = [a,b,c]
#    return np.concatenate(feat_lst)
#get_movie_features(imdb.iloc[0])

#imdb['actors'].apply(rep_col)

In [42]:
# imdb['feat_str'] = imdb['actor1'] + ', ' + imdb['director1'] + ', ' + imdb['country1']
# imdb['feat_str'] = imdb['feat_str'].str.replace(' ','')
# imdb.head()

In [43]:
# imdb['feat_str'][0]

In [44]:
# imdb['movie_feat'] = imdb['feat_str'].apply(rep_col)
# imdb.head(5)

In [45]:
# mov_with_feat['genres'] = mov_with_feat['genres'].str.replace('|',',')
# mov_with_feat['feat_str'] = mov_with_feat['actor1'] + ', ' + mov_with_feat['director1'] + ', ' + mov_with_feat['country1'] #+ ', ' + mov_with_feat['genres']  
# mov_with_feat['feat_str'] = mov_with_feat['feat_str'].str.replace(' ','')
# mov_with_feat.head()

In [46]:
# mov_with_feat['features'] = mov_with_feat['feat_str'].apply(rep_col)
# mov_with_feat[['movieId','features']].head()

In [47]:
# a = mov_with_feat['features'][0].reshape(1,-1)
# for mov_feat in mov_with_feat['features'][1:]:
#     b = mov_feat.reshape(1,-1)
#     a = np.concatenate([a,b])

# a.shape

In [48]:
# np.median(a)

In [49]:
# np.mean(a)

In [50]:
# lista = [sublista.split('|') for sublista in movies['genres']]
# genres = sorted(set([item for s in lista for item in s]))
# dummies = pd.DataFrame(np.zeros((len(movies), len(genres)), dtype=int), columns=genres)
# for i, gen in enumerate(movies['genres']):
#     dummies.loc[i, gen.split('|')] = 0.2
# dummies.values

In [51]:
# dummies.shape

## 4. Obtención de peliculas similares

In [52]:
from sklearn.metrics.pairwise import cosine_similarity

#movie_features = np.concatenate([a,dummies],axis=1)
movie_features = a
cosine_sim = cosine_similarity(movie_features,dense_output=True)
cosine_sim[0:5]

array([[0.9999999 , 0.99860054, 0.9863323 , ..., 0.9622292 , 0.9402538 ,
        0.98196155],
       [0.99860054, 1.        , 0.98280084, ..., 0.9569641 , 0.9364108 ,
        0.97915727],
       [0.9863323 , 0.98280084, 1.        , ..., 0.9870677 , 0.9173041 ,
        0.99051946],
       [0.9770453 , 0.9719447 , 0.99257326, ..., 0.99360794, 0.91474533,
        0.9849532 ],
       [0.9866722 , 0.9812299 , 0.9949675 , ..., 0.9869387 , 0.94054   ,
        0.99644995]], dtype=float32)

In [53]:
# movies.loc[movies['movieId'] == 1,'title'].values[0]

In [54]:
results = {}

for idx, row in movies.iterrows():
    similar_indices = cosine_sim[idx].argsort()[:-100:-1]
    similar_items = [(cosine_sim[idx][i], movies['movieId'][i]) for i in similar_indices]
    results[row['movieId']] = similar_items[1:]

## 5. Recomendar películas similares a una película en particular

In [55]:
def item(mid):
    return movies.loc[movies['movieId'] == mid]['title'].tolist()[0].split(' - ')[0]

def recommend_movie(item_id, num):
    print("Recomendando " + str(num) + " productos similares a " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recomendado: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")
        
#Ingresando el ID de la película y el número de recomendaciones devuelve los títulos mejor posisionados

In [56]:
recommend_movie(1,10)

Recomendando 10 productos similares a Toy Story (1995)...
-------
Recomendado: Monsters, Inc. (2001) (score:0.99969095)
Recomendado: Ice Age 2: The Meltdown (2006) (score:0.99962115)
Recomendado: Antz (1998) (score:0.99962026)
Recomendado: Madagascar (2005) (score:0.99959767)
Recomendado: Horton Hears a Who! (2008) (score:0.9995464)
Recomendado: Toy Story 2 (1999) (score:0.99940795)
Recomendado: The Boss Baby (2017) (score:0.99940354)
Recomendado: Minions (2015) (score:0.99939156)
Recomendado: Emperor's New Groove, The (2000) (score:0.99938977)
Recomendado: Cool World (1992) (score:0.99925184)


In [57]:
recommend_movie(648,10)

Recomendando 10 productos similares a Mission: Impossible (1996)...
-------
Recomendado: Death Proof (2007) (score:0.99957335)
Recomendado: Blade Runner (1982) (score:0.99951583)
Recomendado: Island, The (2005) (score:0.9993232)
Recomendado: Alien³ (a.k.a. Alien 3) (1992) (score:0.99929833)
Recomendado: Thing, The (1982) (score:0.9991589)
Recomendado: One, The (2001) (score:0.9990476)
Recomendado: Matrix, The (1999) (score:0.99904716)
Recomendado: Carrie (1976) (score:0.9990454)
Recomendado: Saint, The (1997) (score:0.9989708)
Recomendado: End of Days (1999) (score:0.9989693)


In [58]:
recommend_movie(2492,10)

Recomendando 10 productos similares a 20 Dates (1998)...
-------
Recomendado: Lola Versus (2012) (score:0.99990004)
Recomendado: Drive Me Crazy (1999) (score:0.9998944)
Recomendado: Lovesick (2014) (score:0.9998915)
Recomendado: Two Night Stand (2014) (score:0.9998842)
Recomendado: Kissing Jessica Stein (2001) (score:0.9998736)
Recomendado: Outsourced (2006) (score:0.9998688)
Recomendado: Allnighter, The (1987) (score:0.99986404)
Recomendado: Amy's O (a.k.a. Amy's Orgasm) (2001) (score:0.9998479)
Recomendado: Lost & Found (1999) (score:0.99983966)
Recomendado: Play the Game (2009) (score:0.9998366)


In [59]:
# imdb['actors'].apply(lambda x: len(x.split(','))).unique()

### Lista de películas vistas por usuario

In [60]:
ratings_sorted = ratings.sort_values(by='timestamp')
movies_by_user = pd.DataFrame(ratings_sorted.groupby('userId')['movieId'].apply(list))
movies_by_user.head()

,movieId
userId,
1,"[804, 1210, 2628, 2826, 2018, 3578, 3617, 3744..."
2,"[318, 79132, 131724, 115713, 99114, 112552, 35..."
3,"[1275, 2105, 1371, 914, 1263, 2080, 31, 720, 3..."
4,"[171, 902, 2351, 417, 1304, 2959, 2712, 162, 5..."
5,"[590, 592, 150, 380, 296, 457, 588, 153, 344, ..."


Para cada usuario me quedo con la última película que vio:

In [61]:
movies_by_user['movieId'][1][-1]

2492

In [71]:
def get_user_views(uid,num):
    movies_ids = movies_by_user.loc[uid]['movieId'][-num:]
    titles = []
    for mid in movies_ids:
        titles.append((mid,movies.loc[movies['movieId']==mid,'title'].values[0]))
    return titles
get_user_views(1,5)

[(1445, "McHale's Navy (1997)"),
 (553, 'Tombstone (1993)'),
 (2478, '¡Three Amigos! (1986)'),
 (2012, 'Back to the Future Part III (1990)'),
 (2492, '20 Dates (1998)')]

In [64]:
movies_by_user.loc[1]['movieId'][-3:]

[2478, 2012, 2492]

In [59]:
movies_by_user['last_watched'] = movies_by_user['movieId'].apply(lambda x: x[-1])
movies_by_user.head()

,movieId,last_watched
userId,,
1,"[804, 1210, 2628, 2826, 2018, 3578, 3617, 3744...",2492
2,"[318, 79132, 131724, 115713, 99114, 112552, 35...",80489
3,"[1275, 2105, 1371, 914, 1263, 2080, 31, 720, 3...",2424
4,"[171, 902, 2351, 417, 1304, 2959, 2712, 162, 5...",4246
5,"[590, 592, 150, 380, 296, 457, 588, 153, 344, ...",474


Y le recomiendo las 10 películas más parecidas a esa, sin recomendarle alguna que ya vio:

In [60]:
def user_recommendation(uid=1,num=1):
    last_watched_id = movies_by_user.loc[uid]['last_watched']
    movies_watched = movies_by_user.loc[uid]['movieId']
    recommendation = results[last_watched_id]#[:num]
    i=0
    j=0
    recs=[]
    while i<num:
        if recommendation[j][1] not in movies_watched:
            recs.append(recommendation[j])
            j+=1
            i+=1
        else:
            j+=1
    
    #print("Recomendando " + str(num) + " productos similares a " + item(last_watched_id) + "...")
    print("Porque ya viste " + item(last_watched_id) + "...")
    print("-------")
    for rec in recs:
        #print("Recomendado: " + str(rec[1]) + ' ' + item(rec[1]) + " (score:" + str(rec[0]) + ")")
        print("Recomendado: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")
    #return recommendation

In [61]:
uid=610
num=10
user_recommendation(uid,num)

Porque ya viste Hellraiser (1987)...
-------
Recomendado: Hellbound: Hellraiser II (1988) (score:0.99960595)
Recomendado: The Plague of the Zombies (1966) (score:0.9994568)
Recomendado: Horror of Dracula (Dracula) (1958) (score:0.998719)
Recomendado: Awakening, The (1980) (score:0.9984838)
Recomendado: Curse of Frankenstein, The (1957) (score:0.9983713)
Recomendado: Mummy, The (1959) (score:0.9983321)
Recomendado: Black Mirror: White Christmas (2014) (score:0.9973811)
Recomendado: Legend of Hell House, The (1973) (score:0.99707943)
Recomendado: The Hound of the Baskervilles (1988) (score:0.9970715)
Recomendado: The Deadly Bees (1967) (score:0.9966915)


In [62]:
uid=1
num=10
user_recommendation(uid,num)

Porque ya viste 20 Dates (1998)...
-------
Recomendado: The Escort (2015) (score:0.99990624)
Recomendado: Outsourced (2006) (score:0.9999056)
Recomendado: Lola Versus (2012) (score:0.99989456)
Recomendado: How to Make Love to a Woman (2010) (score:0.9998936)
Recomendado: It Happened on Fifth Avenue (1947) (score:0.9998914)
Recomendado: Mischief (1985) (score:0.9998781)
Recomendado: Play the Game (2009) (score:0.9998723)
Recomendado: Nina Takes a Lover (1994) (score:0.99985975)
Recomendado: Sex Ed (2014) (score:0.9998594)
Recomendado: Lost & Found (1999) (score:0.99985415)


In [63]:
uid=154
num=10
user_recommendation(uid,num)

Porque ya viste Finding Dory (2016)...
-------
Recomendado: Planes (2013) (score:0.9998964)
Recomendado: ParaNorman (2012) (score:0.9997401)
Recomendado: Dragons: Gift of the Night Fury (2011) (score:0.99951994)
Recomendado: Cars 3 (2017) (score:0.9994921)
Recomendado: Planes: Fire & Rescue (2014) (score:0.9993984)
Recomendado: Puss in Boots: The Three Diablos (2012) (score:0.99934965)
Recomendado: Wreck-It Ralph (2012) (score:0.9992212)
Recomendado: Ice Age 4: Continental Drift (2012) (score:0.99920255)
Recomendado: Black Knight (2001) (score:0.99919)
Recomendado: Alvin and the Chipmunks: Chipwrecked (2011) (score:0.99895793)


In [64]:
# sorted(movies_by_user.loc[uid]['movieId'])

User Random Recommendation (tomo una pelicula random que vio, y le recomiendo otras):

In [66]:
import random

def user_rand_recommendation(uid=1,num=1):
    movies_watched = movies_by_user.loc[uid]['movieId']
    rand_watched_id = random.choice(movies_watched)
    recommendation = results[rand_watched_id]#[:num]
    i=0
    j=0
    recs=[]
    while i<num:
        if recommendation[j][1] not in movies_watched:
            recs.append(recommendation[j])
            j+=1
            i+=1
        else:
            j+=1
    
    #print("Recomendando " + str(num) + " productos similares a " + item(last_watched_id) + "...")
    print("Porque ya viste " + item(rand_watched_id) + "...")
    print("-------")
    for rec in recs:
        #print("Recomendado: " + str(rec[1]) + ' ' + item(rec[1]) + " (score:" + str(rec[0]) + ")")
        print("Recomendado: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")
    #return recommendation

In [72]:
uid=610
num=10
user_rand_recommendation(uid,num)

Porque ya viste Field in England, A (2013)...
-------
Recomendado: Paperhouse (1988) (score:0.99751496)
Recomendado: I Know Where I'm Going! (1945) (score:0.9970697)
Recomendado: Dam Busters, The (1955) (score:0.9967695)
Recomendado: Dorian Gray (2009) (score:0.9967139)
Recomendado: Red Shoes, The (1948) (score:0.9967045)
Recomendado: Henry V (1989) (score:0.9964687)
Recomendado: Pink Floyd: The Wall (1982) (score:0.99643403)
Recomendado: Abominable Snowman, The (Abominable Snowman of the Himalayas, The) (1957) (score:0.9963002)
Recomendado: Road to Guantanamo, The (2006) (score:0.9960692)
Recomendado: Edge of Love, The (2008) (score:0.99591374)


In [73]:
uid=1
num=10
user_rand_recommendation(uid,num)

Porque ya viste That Thing You Do! (1996)...
-------
Recomendado: Unstrung Heroes (1995) (score:0.9993211)
Recomendado: The Disaster Artist (2017) (score:0.99873585)
Recomendado: Bronx Tale, A (1993) (score:0.9984861)
Recomendado: Pollock (2000) (score:0.9983051)
Recomendado: Antwone Fisher (2002) (score:0.9982897)
Recomendado: Man Without a Face, The (1993) (score:0.9982252)
Recomendado: Larry Crowne (2011) (score:0.9981845)
Recomendado: Mr. Saturday Night (1992) (score:0.99787503)
Recomendado: Sling Blade (1996) (score:0.9971619)
Recomendado: Rocky Balboa (2006) (score:0.9971091)


In [74]:
uid=154
num=10
user_rand_recommendation(uid,num)

Porque ya viste Finding Dory (2016)...
-------
Recomendado: Planes (2013) (score:0.9998964)
Recomendado: ParaNorman (2012) (score:0.9997401)
Recomendado: Dragons: Gift of the Night Fury (2011) (score:0.99951994)
Recomendado: Cars 3 (2017) (score:0.9994921)
Recomendado: Planes: Fire & Rescue (2014) (score:0.9993984)
Recomendado: Puss in Boots: The Three Diablos (2012) (score:0.99934965)
Recomendado: Wreck-It Ralph (2012) (score:0.9992212)
Recomendado: Ice Age 4: Continental Drift (2012) (score:0.99920255)
Recomendado: Black Knight (2001) (score:0.99919)
Recomendado: Alvin and the Chipmunks: Chipwrecked (2011) (score:0.99895793)
